In [69]:
import json
from IPython.display import display, HTML
import textile
import collections
import pandas as pd

data = json.load(open('output.json'))
"""{
        "prompt": "Below is a multipart question:\nOn January 2, Year 1, West Co. issued 9% bonds in the amount of $500,000, which mature on January 2, Year 11. The bonds were issued for $469,500 to yield 10%. Interest is payable annually on December 31. West uses the effective interest method of amortizing bond discount. In its June 30, Year 1 balance sheet what amount should West report as bonds payable?\n1. $469,500\n2. $470,475\n3. $471,025\n4. $500,000\n\nAnswer the question by entering the number of the correct option. Response with a single number and nothing else.",
        "full": {
            "replicate/mistral-7b": "3. $471,025",
            "openai/gpt-3.5-turbo-1106": "2",
            "anthropic/claude-2": " 2. $470,475"
        },
        "simplified": {
            "replicate/mistral-7b": "3",
            "openai/gpt-3.5-turbo-1106": "2",
            "anthropic/claude-2": "2"
        },
        "source_question": {
            "category": "FAR",
            "description": "On January 2, Year 1, West Co. issued 9% bonds in the amount of $500,000, which mature on January 2, Year 11. The bonds were issued for $469,500 to yield 10%. Interest is payable annually on December 31. West uses the effective interest method of amortizing bond discount. In its June 30, Year 1 balance sheet what amount should West report as bonds payable?",
            "options": [
                "$469,500",
                "$470,475",
                "$471,025",
                "$500,000"
            ],
            "correct_answer": 2,
            "reference": "BECKER 00470",
            "tag": "BOND"
        }
"""
# Let's visualize the data
# Make a table for each question and model answers
output = []
scorecard = {}
for idx, item in enumerate(data):
    output.append(f'<b style="text-decoration: underline;">Question {idx+1}</b>')
    output.append(item['source_question']['description'])
    correct_answer = int(item['source_question']['correct_answer'])
    for i, option in enumerate(item['source_question']['options']):
        output.append(f'{i+1}. {option}')
    
    for model in item['full']:
        correct = '✅' if int(item['simplified'][model]) == correct_answer else '❌'
        output.append(f'{model}: "{item["full"][model][:20]}" ({item["simplified"][model]}) {correct}')
        if correct == '✅':
            scorecard[model] = scorecard.get(model, 0) + 1
        else:
            scorecard[model] = scorecard.get(model, 0)
    output.append(f'Correct: {correct_answer}')
    output.append('<br>')

df = pd.DataFrame.from_dict(scorecard, orient='index', columns=['Correct'])
# format as percent
df['Score'] = df['Correct'] / len(data)
df['Model'] = df.index
# label index column
del df['Correct']
df = df.sort_values('Score', ascending=False)
df.index = range(1, len(df)+1)
df = df[['Model', 'Score']]
df['Score'] = df['Score'].map('{:,.1%}'.format)
display(df)
display(HTML('<br>'.join(output)))

,Model,Score
1,openai/gpt-4-1106-preview,56.9%
2,anthropic/claude-2,42.6%
3,openai/gpt-3.5-turbo-1106,40.7%
4,replicate/mistral-7b,30.6%
